In [10]:
import pandas as pd
import matplotlib.pyplot as plt

# Tratamento de dados

## Carregamento e analises iniciais 

In [ ]:
df = pd.read_csv('cota-parlamentar.csv')
df.head() #visualiza os primeiros itens do data frame

In [ ]:
df.info() #mostra os tipos de dados de cada coluna

In [ ]:
df.describe() #mostra números ref as colunas

## Selecionando as colunas mais relevantes para a análise

In [ ]:
df_relevante = df[['txnomeparlamentar', 'sgpartido', 'sguf', 'datemissao', 'vlrdocumento', 'vlrglosa', 'vlrliquido', 'txtfornecedor', 'txtdescricao', 'numano']]
df_relevante.head()

## Tratar valores nulos

In [ ]:
# Verificar valores nulos
print(df_relevante.isnull().sum())

In [ ]:
# Preencher valores ausentes de 'sguf' com 'Desconhecido'
df_relevante['sguf'] = df_relevante['sguf'].fillna('Desconhecido')

# 2. Remover linhas com 'datemissao' nulo
df_relevante = df_relevante.dropna(subset=['datemissao'])

# 3. Remover linhas com valores nulos em 'vlrdocumento' ou 'txtfornecedor'
df_relevante = df_relevante.dropna(subset=['vlrdocumento', 'txtfornecedor'])

# Exibir a contagem de valores nulos após o tratamento
print("\nValores nulos após tratamento:")
print(df_relevante.isnull().sum())

## Remover duplicatas (se necessário)

In [30]:
df_relevante = df_relevante.drop_duplicates()

## Converter a coluna 'datemissao' para o tipo datetime

In [31]:
df_relevante['datemissao'] = pd.to_datetime(df_relevante['datemissao'], errors='coerce')

## Verificar se a coluna de valores numéricos está correta


In [32]:
df_relevante[['vlrdocumento', 'vlrglosa', 'vlrliquido']] = df_relevante[['vlrdocumento', 'vlrglosa', 'vlrliquido']].apply(pd.to_numeric, errors='coerce')

# Analise de dados

### 1. Gasto total por parlamentar.

o código para calcular o total gasto por cada parlamentar, agrupando pelo nome e somando os valores da coluna vlrliquido

In [ ]:
# Gasto total por parlamentar
gasto_por_parlamentar = df_relevante.groupby('txnomeparlamentar')['vlrliquido'].sum().reset_index()

# Ordenar os parlamentares pelos maiores gastos
gasto_por_parlamentar = gasto_por_parlamentar.sort_values(by='vlrliquido', ascending=False)

# Exibir os 10 parlamentares que mais gastaram
print(gasto_por_parlamentar.head(20))


In [ ]:


# Selecionar os 10 parlamentares que mais gastaram
top_20_parlamentares = gasto_por_parlamentar.head(20)

# Criar gráfico de barras
plt.figure(figsize=(10,6))
plt.barh(top_20_parlamentares['txnomeparlamentar'], top_20_parlamentares['vlrliquido'], color='skyblue')
plt.xlabel('Gasto Total (R$)')
plt.ylabel('Parlamentar')
plt.title('Top 20 Parlamentares por Gasto Total')
plt.gca().invert_yaxis()  # Inverter eixo para mostrar o maior no topo
plt.show()


### 2. evolução dos gastos ao longo do tempo.

In [ ]:
#Código para agrupar os dados por mês e ano:

# Garantir que a coluna 'datemissao' está no formato datetime
df_relevante['datemissao'] = pd.to_datetime(df_relevante['datemissao'], errors='coerce')

# Criar uma nova coluna 'ano_mes' para agrupar por ano e mês
df_relevante['ano_mes'] = df_relevante['datemissao'].dt.to_period('M')

# Agrupar o gasto total por mês e ano
gastos_por_mes = df_relevante.groupby('ano_mes')['vlrliquido'].sum().reset_index()

# Exibir os primeiros resultados
print(gastos_por_mes.head())


Remover anos com poucos gastos: Vamos filtrar os anos com poucos gastos para melhorar a visualização.

Agrupar os dados por ano: Isso simplifica a visualização e destaca a tendência anual.

In [ ]:
# Garantir que a coluna 'datemissao' está no formato datetime
df_relevante['datemissao'] = pd.to_datetime(df_relevante['datemissao'], errors='coerce')

# Criar uma nova coluna 'ano' para agrupar por ano
df_relevante['ano'] = df_relevante['datemissao'].dt.year

# Filtrar os dados apenas para o período entre 2009 e 2019
df_periodo = df_relevante[(df_relevante['ano'] >= 2009) & (df_relevante['ano'] <= 2019)]

# Agrupar o gasto total por ano
gastos_por_ano = df_periodo.groupby('ano')['vlrliquido'].sum().reset_index()

# Criar o gráfico de linha
plt.figure(figsize=(10,6))
plt.plot(gastos_por_ano['ano'], gastos_por_ano['vlrliquido'], marker='o', linestyle='-', color='b', label='Total de Gastos')

# Ajustar o eixo x para marcar de 3 em 3 anos
plt.xticks(ticks=range(2009, 2020, 3))

# Rótulos e título
plt.xlabel('Ano')
plt.ylabel('Gasto Total (R$)')
plt.title('Evolução dos Gastos Parlamentares de 2009 a 2019')
plt.grid(True)

# Exibir o gráfico
plt.tight_layout()
plt.show()


### 3. Gastos por partido (20 maiores partidos + "Outros")

In [ ]:
# Agrupar gastos por partido
gastos_por_partido = df_periodo.groupby('sgpartido')['vlrliquido'].sum().reset_index()

# Ordenar do maior para o menor gasto
gastos_por_partido = gastos_por_partido.sort_values(by='vlrliquido', ascending=False)

# Separar os 20 maiores partidos
top_20_partidos = gastos_por_partido.head(20)

# Calcular a soma dos "Outros" partidos (aqueles fora dos 20 maiores)
outros_gastos = pd.DataFrame([['Outros', gastos_por_partido.iloc[20:]['vlrliquido'].sum()]], columns=['sgpartido', 'vlrliquido'])

# Concatenar os 20 maiores partidos com a categoria "Outros"
gastos_top_20 = pd.concat([top_20_partidos, outros_gastos])

# Criar o gráfico de barras empilhadas
plt.figure(figsize=(12,7))
plt.bar(gastos_top_20['sgpartido'], gastos_top_20['vlrliquido'], color='skyblue')

# Rótulos e título
plt.xlabel('Partido')
plt.ylabel('Gasto Total (R$)')
plt.title('Gastos Totais por Partido (20 Maiores e Outros) de 2009 a 2019')
plt.xticks(rotation=45)
plt.grid(True)

# Exibir o gráfico
plt.tight_layout()
plt.show()


### 4. Principais categorias de gastos

In [ ]:
# Verificar se o nome correto da coluna de descrição é 'txtdescricao' ou outro
# Caso seja diferente, use o nome correto listado ao verificar as colunas
gastos_por_categoria = df_periodo.groupby('txtdescricao')['vlrliquido'].sum().reset_index()

# Ordenar do maior para o menor gasto e pegar as 10 principais categorias
gastos_top_categorias = gastos_por_categoria.sort_values(by='vlrliquido', ascending=False).head(10)
gastos_top_categorias = gastos_top_categorias.sort_values(by='vlrliquido', ascending=True)

# Criar o gráfico de barras horizontais
plt.figure(figsize=(10,6))
plt.barh(gastos_top_categorias['txtdescricao'], gastos_top_categorias['vlrliquido'], color='lightcoral')

# Rótulos e título
plt.xlabel('Gasto Total (R$)')
plt.ylabel('Categoria de Gasto')
plt.title('Principais Categorias de Gastos Parlamentares de 2009 a 2019')
plt.grid(True)

# Exibir o gráfico
plt.tight_layout()
plt.show()


### 5. Média por parlamentar

In [ ]:
# Agrupar os dados por parlamentar e somar os gastos totais
gastos_por_parlamentar = df_periodo.groupby('txnomeparlamentar')['vlrliquido'].sum().reset_index()

# Contar o número de anos únicos de atuação de cada parlamentar
anos_por_parlamentar = df_periodo.groupby('txnomeparlamentar')['numano'].nunique().reset_index()

# Renomear a coluna para facilitar o merge
anos_por_parlamentar.rename(columns={'numano': 'anos_unicos'}, inplace=True)

# Mesclar o total de gastos com o número de anos únicos
gastos_e_anos = pd.merge(gastos_por_parlamentar, anos_por_parlamentar, on='txnomeparlamentar')

# Calcular a média de gastos por parlamentar (gasto total / anos únicos)
gastos_e_anos['media_gastos_por_ano'] = gastos_e_anos['vlrliquido'] / gastos_e_anos['anos_unicos']

# Ordenar pelo gasto médio
gastos_e_anos = gastos_e_anos.sort_values(by='media_gastos_por_ano', ascending=False).head(10)

# Exibir as primeiras linhas do DataFrame
print(gastos_e_anos.head())


In [ ]:

# Criar gráfico de barras
plt.figure(figsize=(10,6))
plt.barh(gastos_e_anos['txnomeparlamentar'], gastos_e_anos['media_gastos_por_ano'], color='skyblue')
plt.xlabel('Média de Gastos (R$)')
plt.ylabel('Parlamentar')
plt.title('Top 10 Parlamentares por média de Gastos por ano')
plt.gca().invert_yaxis()  # Inverter eixo para mostrar o maior no topo
plt.show()

### 6. Gastos de um parlamentar

Nesta parte você pode olhar de forma mais focada os dados e escolher um parlamentar e o ano para ver os gastos por categoria 

In [ ]:
# Função para plotar os gastos de um parlamentar por categoria em um ano específico
def grafico_gastos_parlamentar_por_categoria(df):
    # Solicitar o nome do parlamentar ao usuário
    nome_parlamentar = input("Digite o nome do parlamentar: ")

    # Solicitar o ano de interesse ao usuário
    ano = input("Digite o ano de interesse (Ex: 2019): ")

    # Filtrar os dados para o parlamentar e o ano escolhidos
    df_filtrado = df[(df['txnomeparlamentar'].str.contains(nome_parlamentar, case=False, na=False)) &
                     (df['numano'] == int(ano))]

    # Verificar se há dados para o parlamentar e o ano escolhidos
    if df_filtrado.empty:
        print(f"Nenhum dado encontrado para o parlamentar '{nome_parlamentar}' no ano {ano}.")
        return

    # Agrupar os gastos por categoria de despesa
    gastos_por_categoria = df_filtrado.groupby('txtdescricao')['vlrliquido'].sum().reset_index()

    # Ordenar do maior para o menor gasto
    gastos_por_categoria = gastos_por_categoria.sort_values(by='vlrliquido', ascending=False)

    # Plotar o gráfico de barras
    plt.figure(figsize=(12,6))
    plt.barh(gastos_por_categoria['txtdescricao'], gastos_por_categoria['vlrliquido'], color='skyblue')

    # Rótulos e título
    plt.xlabel('Gastos (R$)')
    plt.ylabel('Categoria de Despesa')
    plt.title(f'Gastos do parlamentar {nome_parlamentar} por categoria no ano {ano}')
    plt.grid(True)

    # Exibir o gráfico
    plt.tight_layout()
    plt.show()

# Chamar a função passando o DataFrame filtrado por período
grafico_gastos_parlamentar_por_categoria(df_periodo)
